<a href="https://colab.research.google.com/github/AnnaPikuleva/-Alternative-power-plants/blob/main/Kaggle_taxi_precision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install pysal

In [3]:
import warnings
warnings.filterwarnings('ignore')

import requests

import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pysal.viz import mapclassify
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

TEST

In [12]:
! gdown 1_o-jT_a_hEpG8Sw_OQySsGB7cGQLpntK

Downloading...
From: https://drive.google.com/uc?id=1_o-jT_a_hEpG8Sw_OQySsGB7cGQLpntK
To: /content/test.csv
100% 983k/983k [00:00<00:00, 127MB/s]


TRAIN

https://drive.google.com/file/d/1YXhx33LcbU9N0orl9w3H5XexnPLEHzVB/view?usp=sharing

In [13]:
! gdown 1YXhx33LcbU9N0orl9w3H5XexnPLEHzVB

Downloading...
From: https://drive.google.com/uc?id=1YXhx33LcbU9N0orl9w3H5XexnPLEHzVB
To: /content/train.csv
100% 5.70G/5.70G [00:36<00:00, 155MB/s]


In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df = df[['key','pickup_longitude', 'pickup_latitude',
         'dropoff_longitude', 'dropoff_latitude','fare_amount']]

In [ ]:
df.head()

df = df[['key','pickup_longitude', 'pickup_latitude',
         'dropoff_longitude', 'dropoff_latitude','fare_amount']]

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
plt.scatter(df.pickup_latitude, df.pickup_longitude)

In [ ]:
plt.scatter(df.dropoff_latitude, df.dropoff_longitude)

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
lon_min = -74.103095
lat_min = 40.597399
lon_max = -73.729243
lat_max = 40.906114

ny_bbox = lat_min,lon_min,lat_max,lon_max

In [ ]:
df = df[(df.pickup_longitude > lon_min)&
   (df.pickup_longitude < lon_max)&
   (df.pickup_latitude > lat_min)&
   (df.pickup_latitude < lat_max)&
   (df.dropoff_longitude > lon_min)&
   (df.dropoff_longitude < lon_max)&
   (df.dropoff_latitude > lat_min)&
   (df.dropoff_latitude < lat_max)].reset_index(drop=True)

In [ ]:
plt.scatter(df.pickup_longitude, df.pickup_latitude, alpha=.01)

In [ ]:
import folium
from folium.plugins import HeatMap

m = folium.Map(location=[40.730610, -73.935242], 
               zoom_start=11, 
               tiles='cartodbpositron')

heat_data = [[row['pickup_latitude'],row['pickup_longitude']] for index, row in df.iterrows()]


HeatMap(heat_data,
        radius=10).add_to(m)

m

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [ ]:
from geopy import distance

In [ ]:
df['gc_distance'] = [distance.great_circle((x1,y1),(x2,y2)).m for x1,y1,x2,y2 in zip(df.pickup_latitude,
                                                                           df.pickup_longitude,
                                                                          df.dropoff_latitude,
                                                                          df.pickup_longitude)]

In [ ]:
df['geo_distance'] = [distance.geodesic((x1,y1),(x2,y2)).m for x1,y1,x2,y2 in zip(df.pickup_latitude,
                                                                           df.pickup_longitude,
                                                                          df.dropoff_latitude,
                                                                          df.pickup_longitude)]

In [ ]:
df

In [ ]:
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

In [ ]:
print('Great circle error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[['gc_distance']],y),
                        X[['gc_distance']], y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')

In [ ]:
print('Geodesic error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[['geo_distance']],y),
                        X[['geo_distance']], y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')

##OSMNX

Посчитаем расстояние по графу

In [ ]:
import networkx as nx
import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

In [ ]:
G = ox.graph_from_bbox(lat_max, lat_min, lon_max, lon_min,
                        network_type='drive')

In [ ]:
ox.plot_graph(G, fig_height=20, fig_width=20);

In [ ]:
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
gdf_nodes.head()

In [ ]:
from sklearn.neighbors import KDTree
tree = KDTree(gdf_nodes[['y', 'x']], metric='euclidean')

def find_nearest_node(tree, gdf, point):
    
    closest_idx = tree.query([(point.y, point.x)], k=1, return_distance=False)[0]
    nearest_node = gdf.iloc[closest_idx].index.values[0]
    
    return nearest_node

In [ ]:
fig, ax = ox.plot_graph_route(G, route,
                              show=False, close=False, 
                              orig_dest_node_color='red',
                              route_color='green',
                              fig_height=20,
                              fig_width=20)

plt.show()

In [ ]:
m = ox.plot_route_folium(G, 
                         route, 
                         route_color='green', route_opacity=0.7, route_width=3)
m

In [ ]:
def osmnx_route(x1,y1,x2,y2):
    node1 = find_nearest_node(tree, gdf_nodes, Point(x1,y1))
    node2 = find_nearest_node(tree, gdf_nodes, Point(x2,y2))
    
    route = nx.shortest_path(G, node1, node2)
    route = ox.node_list_to_coordinate_lines(G, route)
    return MultiLineString(route)

In [ ]:
routes = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        routes.append(osmnx_route(row.pickup_longitude, row.pickup_latitude,
                       row.dropoff_longitude, row.dropoff_latitude))
    except nx.NetworkXNoPath:
        routes.append(None)
        
df['geometry'] = routes
df = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'})

df = df.dropna()
df = df.to_crs(epsg=32618)

df['route_len'] = df.geometry.length

In [ ]:
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

In [ ]:
print('Great circle error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[['gc_distance']],y),
                        X[['gc_distance']], y, cv=5, scoring='neg_mean_absolute_error')),3)), '$')
print('Road network error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[['route_len']],y),
                        X[['route_len']], y, cv=5, scoring='neg_mean_absolute_error')),3)), '$')

##Spatial relations

In [ ]:
geodesic_buffers = gpd.GeoDataFrame()

geodesic_buffers['geometry'] = gpd.GeoDataFrame(
    geometry=[Point(x) for x in xy]
).apply(lambda x: geodesic_point_buffer(x['geometry'].x, x['geometry'].y, 250), axis=1)

In [ ]:
geodesic_buffers.plot()

In [ ]:
m2 = folium.Map(tiles='cartodbpositron')

gjson = geodesic_buffers.to_json()
polys = folium.features.GeoJson(gjson)

m2.add_children(polys)
m2

Районы города

https://drive.google.com/file/d/1YaToDzd1G25-O7ni5YEf7kw1zO9gQXc4/view?usp=sharing

In [1]:
! gdown 1YaToDzd1G25-O7ni5YEf7kw1zO9gQXc4

Downloading...
From: https://drive.google.com/uc?id=1YaToDzd1G25-O7ni5YEf7kw1zO9gQXc4
To: /content/2010 Neighborhood Tabulation Areas (NTAs).geojson
100% 4.48M/4.48M [00:00<00:00, 62.4MB/s]


In [2]:
nta = gpd.read_file('/content/2010 Neighborhood Tabulation Areas (NTAs).geojson')
nta.head(2)

NameError: ignored

In [ ]:
nta.plot();

In [ ]:
df['geometry'] = [Point(x,y) for x,y in zip (df.pickup_longitude, df.pickup_latitude)]
df = gpd.GeoDataFrame(df)

df = gpd.sjoin(df, nta[['ntacode', 'geometry']], op='within')
df = df.rename(columns={'ntacode':'pickup_nta'})
df = df.drop('index_right', axis=1)

df['geometry'] = [Point(x,y) for x,y in zip (df.dropoff_longitude, df.dropoff_latitude)]
df = gpd.GeoDataFrame(df)

df = gpd.sjoin(df, nta[['ntacode', 'geometry']], op='within')
df = df.rename(columns={'ntacode':'dropoff_nta'})
df = df.drop('index_right', axis=1)
df.head()

In [ ]:
nta = nta.merge(df.groupby('pickup_nta').count()['key'].rename('pickup_cnt').reset_index(),
                left_on='ntacode',
                right_on='pickup_nta').drop('pickup_nta', axis=1)
nta = nta.merge(df.groupby('dropoff_nta').count()['key'].rename('dropoff_cnt').reset_index(),
                left_on='ntacode',
                right_on='dropoff_nta').drop('dropoff_nta', axis=1)

In [ ]:
nta = nta.merge(df.groupby('pickup_nta').mean()['fare_amount'].rename('mean_pickup_fare').reset_index(),
                left_on='ntacode',
               right_on='pickup_nta').drop('pickup_nta', axis=1)
nta = nta.merge(df.groupby('dropoff_nta').mean()['fare_amount'].rename('mean_dropoff_fare').reset_index(),
                left_on='ntacode',
               right_on='dropoff_nta').drop('dropoff_nta', axis=1)

In [ ]:
nta.plot('pickup_cnt',
           figsize=(10, 15),
           cmap='OrRd', 
           scheme='quantiles',
           legend=True);

In [ ]:
nta.plot('pickup_cnt',
           figsize=(10, 15),
           cmap='OrRd', 
           scheme='fisher_jenks',
           legend=True);

In [ ]:
from keplergl import KeplerGl

In [ ]:
kg_map = KeplerGl()
kg_map.add_data(nta)
kg_map

In [ ]:
kg_map2 = KeplerGl()
kg_map2.add_data(df, name="trips")
kg_map2

In [ ]:
df = df.merge(nta[['ntacode','mean_pickup_fare','pickup_cnt']],
              left_on = 'pickup_nta',
              right_on='ntacode').drop('ntacode', axis=1)
df = df.merge(nta[['ntacode', 'mean_dropoff_fare', 'dropoff_cnt']],
              left_on = 'dropoff_nta',
              right_on='ntacode').drop('ntacode', axis=1)

In [ ]:
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

print('Road network error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[['route_len']],y),
                        X[['route_len']], y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')

cols = ['route_len', 'pickup_cnt', 'dropoff_cnt', 'mean_pickup_fare', 'mean_dropoff_fare']

print('Target encoded to NTAs: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[cols],y),
                        X[cols], y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')

##OSM amenity data

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = '''
[out:json];
(node["amenity"]({},{},{},{});
);
out center;
'''.format(*ny_bbox)

response = requests.get(overpass_url, 
                        params={'data': overpass_query})
amenity = response.json()

amenity_osm = pd.DataFrame(amenity['elements'])
amenity_osm = amenity_osm.join(
    pd.DataFrame([x['tags'] for x in amenity['elements']])).drop('tags', axis=1)

In [ ]:
amenity_osm.head(2)

In [ ]:
amenity_osm.amenity.value_counts()[:10]

In [ ]:
amenity_osm['geometry'] = [Point(x,y) for x,y in zip(amenity_osm.lon, amenity_osm.lat)]
amenity_osm = gpd.GeoDataFrame(amenity_osm)

amenity_osm = gpd.sjoin(amenity_osm, nta[['ntacode', 'geometry']], op='within')
amenity_osm.head(2)

In [ ]:
top30_amenity = amenity_osm.amenity.value_counts()[:30].index
amenity_osm = amenity_osm[amenity_osm.amenity.isin(top30_amenity)]

amenity_osm = amenity_osm.groupby(['ntacode','amenity']).size().unstack(fill_value=0).reset_index()

In [ ]:
amenity_osm.head()

In [ ]:
df = pd.merge(df, amenity_osm, left_on='pickup_nta', right_on='ntacode').drop('ntacode', axis=1)
df = pd.merge(df, amenity_osm, left_on='dropoff_nta', right_on='ntacode').drop('ntacode', axis=1)

In [ ]:
df.head()

In [ ]:
X = df.drop(['key', 'fare_amount', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
             'gc_distance', 'geo_distance', 'geometry','pickup_nta', 'dropoff_nta'], axis=1)
y = df['fare_amount']

In [ ]:
cols = ['route_len', 'pickup_cnt', 'dropoff_cnt', 'mean_pickup_fare', 'mean_dropoff_fare']

print('NTA encoded target error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X[cols],y),
                        X[cols], y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')

print('OSM amenity counts error: ', abs(round(np.mean(cross_val_score(XGBRegressor().fit(X,y),
                        X, y, cv=5, scoring='neg_mean_absolute_error')),3)),'$')